# Countries with highest rate of deaths and recovery
> An attempt to depict the countries with highest rate of deaths viz-a-viz countries that have a very good rate of recovery.

- author: Arun Gupta
- categories: [compare, interactive]
- image: images/highest-deaths-and-recovered-rates.png
- permalink: /highest-deaths-and-recovered-rates/

The COVID-19 has hit all the countries around the world. Every country is facing challenges to control the spread and impacts of this virus.

An infection in a person either leads to recovery, or (in some cases) a death. Some countries are facing a high number of deaths whereas other few are having a good, high rate of recovery.

Here is an attempt to depict the countries with highest rate of deaths viz-a-viz countries that have a very good rate of recovery, given a specific number of infected people.

This depiction and comparison can be used as a first reference point to:
1. Learn the good things that were followed by those countries, so that those things may be adapted in other countries too.
2. Identify and avoid the features/things/behaviours from the countries that are facing high rate of deaths.

In [ ]:
#hide
import pandas as pd
import altair as alt
from load_covid_data import load_individual_timeseries

# load data for infections, deaths, and recovered
df_confirmed = load_individual_timeseries('confirmed')
df_death = load_individual_timeseries('deaths')
df_recovered = load_individual_timeseries('recovered')

# clean the data
df_confirmed = df_confirmed[~df_confirmed['country'].str.contains(' \(total\)')].drop(['state', 'type'], axis=1, ).reset_index()
df_death = df_death[~df_death['country'].str.contains(' \(total\)')].drop(['state', 'type'], axis=1, ).reset_index()
df_recovered = df_recovered[~df_recovered['country'].str.contains(' \(total\)')].drop(['state', 'type'], axis=1, ).reset_index()

# get the data only for the latest date
Latest_Date = df_confirmed['date'].max()
df_confirmed = df_confirmed.loc[df_confirmed['date'] == Latest_Date]
df_death = df_death.loc[df_death['date'] == Latest_Date]
df_recovered = df_recovered.loc[df_recovered['date'] == Latest_Date]

# remove column 'date' as it is no more required
df_confirmed = df_confirmed.drop('date', axis=1)
df_death = df_death.drop('date', axis=1)
df_recovered = df_recovered.drop('date', axis=1)

# aggregate data
df_confirmed = (df_confirmed.sort_values(by=['country'])
                            .groupby(['country'])
                            .agg(sum)).reset_index()

df_death = (df_death.sort_values(by=['country'])
                    .groupby(['country'])
                    .agg(sum)).reset_index()

df_recovered = (df_recovered.sort_values(by=['country'])
                            .groupby(['country'])
                            .agg(sum)).reset_index()

# rename the column for 'cases' in respective datasets
df_confirmed = df_confirmed.rename(columns={"cases": "infections"})
df_death = df_death.rename(columns={"cases": "deaths"})
df_recovered = df_recovered.rename(columns={"cases": "recovered"})

# (inner)join the datasets for 'confirmed' and 'death' on 'country'
df_Master = pd.merge(df_confirmed, df_death, how='inner', on='country', left_on=None, right_on=None, left_index=False, right_index=False, sort=True)
df_Master = pd.merge(df_Master, df_recovered, how='inner', on='country', left_on=None, right_on=None, left_index=False, right_index=False, sort=True)

# rename the Countries
df_Master['country'] = df_Master['country'].replace({'Bosnia and Herzegovina':'Bosnia Herzegovina',
                                                     'Timor-Leste'           :'East Timor',
                                                     "Cote d'Ivoire"         :'Ivory Coast',
                                                     'Burma'                 :'Myanmar',
                                                     'Korea, South'          :'South Korea',
                                                     'Taiwan*'               :'Taiwan',
                                                     'US'                    :'United States of America',
                                                     'Holy See'              :'Vatican City'})

## remove the countries that have less than 200 confirmed cases
case_threshold = 200 
keep_countries = df_Master.loc[(df_Master['infections'] > case_threshold)].country
df_Master = df_Master.loc[df_Master['country'].isin(keep_countries)]

# calculate 'deaths' and 'recovery' per 1000 infections
df_Master['Deaths_per_1000'] = round(1000 * (df_Master['deaths']/df_Master['infections']), 2)
df_Master['Recovered_per_1000'] = round(1000 * (df_Master['recovered']/df_Master['infections']), 2)

# get countries with top 50 death rates
df_top_death = df_Master.sort_values(by=['Deaths_per_1000'], ascending=False)
df_top_death = df_top_death.head(50)

# get countries with top 50 recovery rates
df_top_recovered = df_Master.sort_values(by=['Recovered_per_1000'], ascending=False)
df_top_recovered = df_top_recovered.head(50)

# get rest of the countries
df_ROW = df_Master.loc[~df_Master['country'].isin(df_top_death.country)]
df_ROW = df_ROW.loc[~df_ROW['country'].isin(df_top_recovered.country)]

C:\Users\GUPTAAX32\OpenSourceContri\load_covid_data.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  .reset_index(level=['country', 'type']))


This graph shows the 50 countries that are facing highest rate of deaths per 1000 infections, across the world. As it is not so intuitive that the developed countries such as Belgium, France, Netherlands are among the top countries having highest death rate.

In [ ]:
#hide_input
# make the bar-chart for countries on "deaths per 1000 infections"
alt.Chart(df_top_death).mark_bar(color='Orange').encode(
    x= alt.Y('country:N', sort='-y', title="Countries"),
    y=alt.Y('Deaths_per_1000', title="Deaths per 1000 infected people")
).properties(
    title='Countries with top death rates')

Further, we move on to the countries with best recovery rate. The graph below shows 50 countries with highest rate of recovery.

In [ ]:
#hide_input
# make the bar-chart for countries on "recovery per 1000 infections"
alt.Chart(df_top_recovered).mark_bar(color='#00CC66').encode(
    x= alt.Y('country:N', sort='-y', title="Countries"),
    y=alt.Y('Recovered_per_1000', title="Recovered per 1000 infected people")
).properties(
    title='Countries with top recovery rates')

Finally we project the data from above graphs on to a scatter plot. The Orange and Green bubbles are from the above two bar charts. Additionally, the bubbles in Grey color are the rest of the countries.

In [ ]:
#hide_input
# make the scatter plot for "Death Rates by Population Density"
chart_high_recovery = alt.Chart(df_top_recovered).mark_circle(size=150, color='#00CC66').encode(
    x=alt.Y('Recovered_per_1000',  title="Recovery per 1000 infections"),
    y=alt.Y('Deaths_per_1000',  title="Deaths per 1000 infections"),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

chart_high_death = alt.Chart(df_top_death).mark_circle(size=150, color='Orange').encode(
    x=alt.Y('Recovered_per_1000',  title="Recovery per 1000 infections"),
    y=alt.Y('Deaths_per_1000',  title="Deaths per 1000 infections"),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

chart_rest = alt.Chart(df_ROW).mark_circle(size=150, color='#C3C3C3').encode(
    x=alt.Y('Recovered_per_1000'),
    y=alt.Y('Deaths_per_1000'),
    tooltip=['country', 'Recovered_per_1000', 'Deaths_per_1000']
).properties(
    width=700,
    height=450)

(chart_high_recovery + chart_high_death + chart_rest).interactive()

The visualizations were made by [Arun Gupta](https://www.linkedin.com/in/arungupta21/), with thanks to [*Johns Hopkins University CSSE*](https://systems.jhu.edu/) for the [*2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository*](https://github.com/CSSEGISandData/COVID-19).